# Project imports

In [1]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import pytorch_lightning as pl


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


# Data Loading step

In [10]:
"""
Create data loaders and augmentations needed here
"""
from Data.ShapeNetDataLoader import ShapeNetVoxelData
from Utils.visualization import visualize_occupancy

overfit = True

shapenet_core_path = Path("D:\ShapeNetCore.v2")
shapenet_splits_csv_path = Path("Data/shapenet_splits.csv")
train_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="train", overfit=overfit)
val_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="val", overfit= overfit)
test_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="test", overfit=overfit)

train_sample = train_data[7]
print(f'Voxel Dimensions: {train_sample.shape}')
print(f"Test Set Size: {len(train_data)}")

print(train_sample)
visualize_occupancy(train_sample.squeeze(), flip_axes=True)

Voxel Dimensions: (1, 128, 128, 128)
Test Set Size: 64
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
  

Output()

# Reconstruction Networks

In [10]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""
from Networks.Image2Mesh import Image2Voxel

model = Image2Voxel(dataloader,dataloader,dataloader,device)

X = torch.rand(10,3,32,32)
pred = model(X)

NameError: name 'dataloader' is not defined

In [7]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else None,
    profiler=True,
    progress_bar_refresh_rate=15
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 67.1 M


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s]                        

F:\Python\Anaconda\envs\ml3d\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


Epoch 14: 100%|██████████| 4/4 [00:13<00:00,  3.46s/it, loss=0.483, v_num=16]



Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_fit_start        	|  0.0            	|  0.0            
on_validation_start 	|  0.0019375      	|  0.031          
on_validation_epoch_start	|  0.0            	|  0.0            
on_validation_batch_start	|  0.0            	|  0.0            
validation_step_end 	|  0.0            	|  0.0            
on_validation_batch_end	|  0.0            	|  0.0            
on_validation_epoch_end	|  0.0            	|  0.0            
on_validation_end   	|  5.8144         	|  93.03          
on_train_start      	|  0.016          	|  0.016          
on_epoch_start      	|  0.0010667      	|  0.016          
on_train_epoch_start	|  0.0            	|  0.0            
get_train_batch     	|  0.0037         	|  0.111          
on_batch_start      	|  0.0            	|  0.0            
on_train_batch_start	|  0.0            	|  0.0            
training_ste

1

# Purifying predicted Meshes

In [ ]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

# Mesh Encoding

In [46]:
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""
from Networks.VoxelAutoencoder import VoxelAutoencoder

dim = 128
model = VoxelAutoencoder(dim, train_data, val_data, test_data, device)
x = torch.randn(3, 1, dim, dim, dim)
print(x)

tensor([[[[[-6.7709e-01,  3.1648e-01,  1.4032e-01,  ...,  2.4897e-01,
             2.4570e+00,  1.6135e+00],
           [ 6.9510e-01, -1.2519e+00,  2.8485e-01,  ..., -2.0600e+00,
             7.3370e-01,  1.8864e-01],
           [-1.6289e+00,  6.0752e-01,  3.9304e-01,  ...,  4.8764e-01,
            -1.2195e-01, -1.5041e+00],
           ...,
           [-1.6331e-01, -4.4866e-01,  8.3140e-01,  ..., -3.7218e-01,
             8.8602e-01, -1.2299e+00],
           [-1.6240e-01, -3.4217e-01,  6.5251e-01,  ...,  1.2124e+00,
             1.4586e-01,  2.9035e-01],
           [ 6.7903e-01, -2.0537e+00, -6.2336e-01,  ...,  6.8824e-01,
            -3.6796e-01, -9.1003e-01]],

          [[ 1.8812e-01,  7.7677e-01,  7.7990e-01,  ...,  5.3242e-01,
            -4.6112e-01,  6.9535e-01],
           [-2.2836e-01,  1.4743e+00, -1.2614e+00,  ...,  7.5975e-01,
             7.0962e-01,  2.5904e-02],
           [-5.7293e-01, -7.9508e-01, -7.7342e-01,  ...,  1.4177e+00,
            -6.1747e-01, -1.1676e+00],
 

In [47]:
model.eval()
with torch.no_grad():
    y = model(x)
print(y)
y.shape

tensor([[[[[5.1150e-01, 3.3131e-02, 9.2993e-01,  ..., 2.7815e-01,
            3.9850e-01, 7.0902e-01],
           [9.9360e-01, 2.2752e-06, 9.9991e-01,  ..., 3.8359e-07,
            9.9979e-01, 2.3668e-03],
           [9.9972e-01, 2.3268e-06, 9.5983e-01,  ..., 3.2668e-05,
            3.2748e-03, 7.3824e-01],
           ...,
           [9.7411e-01, 2.3351e-02, 8.7052e-01,  ..., 1.2639e-02,
            9.0639e-01, 8.4266e-02],
           [1.0998e-02, 2.4810e-01, 5.9405e-02,  ..., 8.8293e-03,
            2.7458e-01, 5.9870e-02],
           [7.4787e-01, 2.6326e-01, 8.7238e-01,  ..., 8.8198e-02,
            8.6241e-01, 3.6880e-01]],

          [[6.2924e-02, 2.9335e-01, 9.9987e-01,  ..., 1.6223e-01,
            8.2384e-01, 3.0411e-01],
           [1.3244e-04, 7.4520e-08, 1.6657e-03,  ..., 6.4339e-12,
            7.4002e-08, 8.8688e-04],
           [6.9278e-01, 2.1277e-01, 1.0000e+00,  ..., 6.3201e-01,
            1.0000e+00, 5.4114e-05],
           ...,
           [2.2789e-03, 2.3633e-02, 1.6

torch.Size([3, 1, 128, 128, 128])

In [48]:
trainer = pl.Trainer(
    max_epochs=8,
    gpus=1 if torch.cuda.is_available() else None,
    progress_bar_refresh_rate=2, 
)

trainer.fit(model)

C:\Users\svlwe\miniconda3\envs\ml-3d\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=2)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 8.7 M 
----------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total params
34.957    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\svlwe\miniconda3\envs\ml-3d\lib\site-packages\pytorch_lightning\trainer\data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


C:\Users\svlwe\miniconda3\envs\ml-3d\lib\site-packages\pytorch_lightning\trainer\data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\svlwe\miniconda3\envs\ml-3d\lib\site-packages\pytorch_lightning\trainer\data_loading.py:407: UserWarning: The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s] 

C:\Users\svlwe\miniconda3\envs\ml-3d\lib\site-packages\pytorch_lightning\loops\optimization\closure.py:35: LightningDeprecationWarning: One of the returned values {'train_n_correct', 'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  rank_zero_deprecation(


Epoch 7: 100%|██████████| 8/8 [00:04<00:00,  1.76it/s, loss=0.0136, v_num=10]


In [45]:
train_test_sample = train_data[1]
visualize_occupancy(train_test_sample.squeeze(), flip_axes=True)

tensor_test = torch.from_numpy(train_test_sample[np.newaxis, :])
print(tensor_test.shape)

model.eval()
with torch.no_grad():
    decoded_test = model(tensor_test)

tmp_decoded = decoded_test.clone()
tmp_decoded[decoded_test<0.5] = 0
tmp_decoded[decoded_test>=0.5] = 1

decoded_test_np = tmp_decoded.squeeze().detach().numpy()
# TODO: convert to occupancy grid

visualize_occupancy(decoded_test_np, flip_axes=True)
print(np.sum(decoded_test_np))
#visualize_occupancy(np.ones((32, 32, 32)), flip_axes=True)

Output()

torch.Size([1, 1, 128, 128, 128])


Output()

1005695.0


# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....